<a href="https://colab.research.google.com/github/stac-bot/Pytorch-for-Deep-Learning/blob/main/Quickstart__with_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Task 1: Working with data

At the core of PyTorch data loading utility is torch.utils.data.DataLoader class.<br> What is in it for us? good question? <br>
**Data Loader combines datasets and sampler, and provides and iterable over the given datasets it also works for both map-style and iterable-style datasets**
It has python iterable over the dataset, that support various operations.<br>
1. map-style (protocols:(__getitem__(),__len__()) and iterable-style datasets(__iter__()),

2. customizing data loading order(user defined Sampler- works only with map-style, since iterable-style datasets have no notion of key or an index),

3. automatic batching (take a look at collate_fn https://pytorch.org/docs/stable/data.html#working-with-collate-fn),

4. single- and multi-process data loading (https://pytorch.org/docs/stable/data.html#single-and-multi-process-data-loading),<br> `Note:number of workers * size of parent process`

5. automatic memory pinning (for faster data transfer- https://pytorch.org/docs/stable/data.html#memory-pinning ). 
 
 DatLoader is an constructor which has all of this options


sub-task 1: import domain-specific libraries

In [1]:
import torch 
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


https://www.cs.toronto.edu/~kriz/cifar.html - dive deep into dataset

In [3]:
# download training data from open datasets
training_data = datasets.FashionMNIST(root="data", # path of training data
                                      train = True, # specifies training set or test datasets
                                      download = True, # download the data from internet if not avail locally 
                                      transform = ToTensor(), # specify the feature label transformations
                                      ) 

# download test data from open datasets 
test_data =  datasets.FashionMNIST(root="data",
                                   train="False",
                                   download = True,
                                   transform = ToTensor())

sub-task 2: now we pass our dataset to an argument DataLoader, as we recall which provides iterable over *dataset, and support automatic batching, sampling, shuffling, and multiprocess data loading*

In [4]:
batch_size = 64

# create data loaders.
train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

for X, y in test_dataloader:
  print(f"Shape of X[N, C, H, W]: {X.shape}")
  print(f"Shape of y: {y.shape} {y.dtype}")
  break

Shape of X[N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


##Task 2 : creating Model

to define neural network we always create a class that inherits from nn.Module and then we define layers of networks in the __init__ function further defining how data will pass through the network n Forword function. <br>
for boost our work we move it to GPU if avail 

In [13]:
# Get cpu or gpu device for training
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# defince our model
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10)
    )
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

# get the model on same device
model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Task 3: train the model 

for training the model we need loss function an optimizer

In [19]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [20]:
# define the training loop
def train(dataloader, model,loss_fn, optimizer):
  size = len(dataloader)
  model.train()
  for batch, (X,y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # Compute prediction error
    pred = model(X)
    loss = loss_fn(pred,y)

    #backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")


# define the test loop
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss , correct = 0,0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred,y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")

sub-tak : training process is conducted over several iterations(epochs). What happens here?<br>
Model learns parameters to make better predictions.

In [21]:
epochs = 2 
for t in range(epochs):
  print(f"Epoch {t+1}\n---------------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader,model,loss_fn)
print("Done!")

Epoch 1
---------------------------------
loss: 2.304174 [    0/  938]
loss: 2.145146 [ 6400/  938]
loss: 1.759648 [12800/  938]
loss: 1.497524 [19200/  938]
loss: 1.140288 [25600/  938]
loss: 1.036516 [32000/  938]
loss: 1.005391 [38400/  938]
loss: 0.861935 [44800/  938]
loss: 0.865637 [51200/  938]
loss: 0.803440 [57600/  938]
Test Error: 
 Accuracy: 72.6%, Avg loss: 0.771530

Epoch 2
---------------------------------
loss: 0.788761 [    0/  938]
loss: 0.844542 [ 6400/  938]
loss: 0.574272 [12800/  938]
loss: 0.780884 [19200/  938]
loss: 0.673564 [25600/  938]
loss: 0.647559 [32000/  938]
loss: 0.720643 [38400/  938]
loss: 0.682259 [44800/  938]
loss: 0.706926 [51200/  938]
loss: 0.640689 [57600/  938]
Test Error: 
 Accuracy: 78.9%, Avg loss: 0.612661

Done!


## Saving the model 

In [23]:
torch.save(model.state_dict(),"model.path")
print("saved PyTorch Model State to model.pth")

saved PyTorch Model State to model.pth
